In [ ]:
import glob
import nibabel as nib
import pandas as pd
import numpy as np
import os
import h5py
from sklearn import linear_model
from scipy.stats import zscore
from scipy import stats
import matplotlib.pyplot as plt
import math
import pickle
import joblib
import sys
import deepdish as dd

from modules.make_BIDS_behav_dict import make_BIDS_behav_dict
from modules.get_listenorder import get_listenorder
from modules.listen_timing import listen_timing
from modules.avg_story_lens import avg_story_lens
from modules.create_residuals import create_residuals
from modules.all_voxels_story_data import all_voxels_story_data
from modules.all_ROIs_story_data import all_ROIs_story_data
from modules.process_Recall_Residuals import process_Recall_Residuals
from modules.do_motion_assessment import do_motion_assessment
from modules.create_residuals_no_mo import create_residuals_no_mo

In [ ]:
subs = ['sub-01', 'sub-02', 'sub-03', 'sub-04', 'sub-05', 'sub-06', 'sub-07', 'sub-08', 'sub-09', 'sub-10',
       'sub-12', 'sub-13','sub-14', 'sub-16', 'sub-17', 'sub-18', 'sub-19', 'sub-20', 'sub-21', 'sub-22',
        'sub-23', 'sub-24', 'sub-25', 'sub-26', 'sub-27', 'sub-28', 'sub-29', 'sub-30', 'sub-31', 'sub-32', 'sub-33', 
       'sub-34', 'sub-35', 'sub-36', 'sub-37', 'sub-38']

n_subs = len(subs)

story_lens = pickle.load(open('pickle/story_lens36.pickle', 'rb'))
timing = pickle.load(open('pickle/timing36.pickle', 'rb'))
listen_order = pickle.load(open('pickle/listen_order36.pickle', 'rb'))

In [ ]:
for sub_i, sub in enumerate(subs):
    print('N hipp voxels for ', sub, '...')
    filename = 'pickle/subs_encoding_hipp_storydict_ISC_80/' + sub + '_encoding_hipp_storydict_ISC_80.sav'
    this_sub_storydict = joblib.load(filename)
    if 11 in this_sub_storydict.keys():
        print(this_sub_storydict[11].shape[0])
    elif 12 in this_sub_storydict.keys():
        print(this_sub_storydict[12].shape[0])
    else: 
        print(this_sub_storydict[13].shape[0])

In [ ]:
story_data = dict()


nVox = 785
story_data = dict()
for story in story_lens:
    story_data[story] = np.full((n_subs, nVox, story_lens[story]['ISC']['L'].shape[1]), np.nan)
        
for sub_i, sub in enumerate(subs):
    print('processing...', sub)
    filename = 'pickle/subs_encoding_hipp_storydict_ISC_80/' + sub + '_encoding_hipp_storydict_ISC_80.sav'
    this_sub_storydict = joblib.load(filename)
    for this_story in this_sub_storydict.keys():
        run_data = this_sub_storydict[this_story]
        story_data[this_story][sub_i,:] = run_data

filename = 'pickle/hipp_storydict_ISC_' + str(len(subs)) + '_80.sav'
joblib.dump(story_data, filename)

In [ ]:
# hipp


story_lens = pickle.load(open('pickle/story_lens36.pickle', 'rb'))
timing = pickle.load(open('pickle/timing36.pickle', 'rb'))
listen_order = pickle.load(open('pickle/listen_order36.pickle', 'rb'))



# Loading ROIs
ROIs = ['Ang', 'Aud', 'mPFC', 'PHC', 'PMC', 'SFG', 'STS']
masks = dict()
story_data = dict()

for roi in ROIs:
    with h5py.File('/data/ListEvents/SchemaROIs/' + roi + '_verts.h5', "r") as f:
        masks[roi] = {'L': list(f['left']), 'R':list(f['right'])}
    nVox = len(masks[roi]['L'])+len(masks[roi]['R'])
    story_data[roi] = dict()
    for story in story_lens:
        story_data[roi][story] = np.full((n_subs, nVox, story_lens[story]['ISC']['L'].shape[1]), np.nan)
        
for sub_i, sub in enumerate(subs):
    print('processing...', sub)
    filename = 'pickle/subs_encoding_storydict_ISC/' + sub + '_encoding_storydict_ISC.sav'
    this_sub_storydict = joblib.load(filename)
    for this_story in this_sub_storydict.keys():
        for roi in ROIs:
            run_data = np.row_stack((this_sub_storydict[this_story]['L'][masks[roi]['L']],this_sub_storydict[this_story]['R'][masks[roi]['R']]))
            story_data[roi][this_story][sub_i,:] = run_data

filename = 'pickle/roi_storydict_ISC_' + str(len(subs)) + '.sav'
joblib.dump(story_data, filename)

In [ ]:
plt.imshow(story_data['Ang'][11][:,0,:])